In [1]:
# INTEL CORPORATION CONFIDENTIAL AND PROPRIETARY
# 
# Copyright © 2020-2021 Intel Corporation.
# 
# This software and the related documents are Intel copyrighted
# materials, and your use of them is governed by the express 
# license under which they were provided to you (License). Unless
# the License provides otherwise, you may not use, modify, copy, 
# publish, distribute, disclose or transmit  this software or the
# related documents without Intel's prior written permission.
# 
# This software and the related documents are provided as is, with
# no express or implied warranties, other than those that are 
# expressly stated in the License.

# Approximate KNN search on Loihi
This notebook and module show how to implement the approximate KNN search algorithm described in the paper below:

>E. Paxon Frady, Garrick Orchard, David Florey, Nabil Imam, Ruokun Liu, Joyesh Mishra, Jonathan Tse, Andreas Wild, Friedrich T. Sommer, and Mike Davies. 2020. Neuromorphic Nearest Neighbor Search Using Intel's Pohoiki Springs. In Proceedings of the Neuro-inspired Computational Elements Workshop (NICE '20). Association for Computing Machinery, New York, NY, USA, Article 23, 1–10.  
DOI: https://doi.org/10.1145/3381755.3398695  
PrePrint: https://arxiv.org/abs/2004.12691   

In [2]:
import nxsdk
import time
from nxsdk_modules.knn.src.dimensionReduction import DimensionReduction as DR
from nxsdk_modules.knn.src.knn import Knn
import numpy as np
import os
from nxsdk.logutils.nxlogging import set_verbosity, LoggingLevel
set_verbosity(LoggingLevel.ERROR) #suppress all the compilation warnings

### Specify a Loihi system to run on

In [3]:
# For Nahuku 32
os.environ["PARTITION"]= "nahuku32"
os.environ["BOARD"]= "ncl-ext-ghrd-01" # for external vLab
#os.environ["BOARD"]= "ncl-ghrd-04"     # for internal

# For Pohoiki
#os.environ["PARTITION"]= "pohoiki"
#os.environ["BOARD"]= "ncl-ext-ps-01-dct-01" # for external vLab
#os.environ["BOARD"]= "ncl-ps-01-dct-01"     # for internal

Detect how many chips the chosen Loihi system contains. Later we will use this to truncate the dataset down to the largest size that can fit on the chosen Loihi system.

In [4]:
numChips = !srun --partition={os.environ["PARTITION"]} --nodelist={os.environ["BOARD"]} {nxsdk.__path__[0]}/bin/nx --detect-chips
numChips = int(numChips[len(numChips)-1])
numNahukus = numChips//32
print("{} Nahuku(s) detected".format(numNahukus))

1 Nahuku(s) detected


### Setup the dataset 
Specify the location of the dataset, query, and optional encoding file <br/>
If the dataset folder doesn't exist, download the dataset

In [5]:
datasetFolder = "{}/gist".format(os.getcwd())
encodingFile = '{}/encode_matrix.npy'.format(datasetFolder)
datasetFile = '{}/gist_base.fvecs'.format(datasetFolder)
queryFile = '{}/gist_query.fvecs'.format(datasetFolder)

def readGistData(filename):
    """
    Function for reading dataset format. Zero centers and normalizes each element
    """
    data = np.fromfile(filename, dtype=np.float32)
    data = data.reshape((data.shape[0]//961, -1))
    assert (data.view(np.int32)[:,0] == 960).all(), "unexpected file format"

    data = data[:,1:]
    data -= np.expand_dims(data.mean(axis=1), axis=1)
    data /= np.expand_dims(np.linalg.norm(data, axis=1), axis=1) + np.finfo(float).eps
    return data

## FTP is blocked on the external vlab, so you would need to download the dataset yourself and copy it over!
Download the file on your own system:
> wget ftp://ftp.irisa.fr/local/texmex/corpus/gist.tar.gz  

Then copy to external vlab in the same folder as this notebook
> scp gist.tar.gz (yourname)@(vlab):(thisfolder)

In [6]:
if not os.path.isdir(datasetFolder):
    # If you are not on external vlab, you can uncomment the line below to directly download the dataset
    #!wget ftp://ftp.irisa.fr/local/texmex/corpus/gist.tar.gz 
    
    if not os.path.isfile("{}/gist.tar.gz".format(datasetFolder)):
        print("Error, the dataset file is missing. See instructions above")
        
    !tar -zxvf gist.tar.gz && rm gist.tar.gz

### Setup KNN parameters
| Parameter       | Description                                                                      | Comment
|-----------------|----------------------------------------------------------------------------------|--------------
| K               | The number of approximate nearest neighbours to return                           |
| queryDuration   | Number of Loihi timesteps over which to encoded query as spikes                  | Controls temporal resolution
| spikesPerQuery  | Number of spikes used to encode a query                                          | Each subsequent spike encodes the next largest dimension
| neuronThreshold | The sensitivity of the neurons                                                   | 
| benchmark       | Tells the lakemonts whether to gather and return per-timestep timing information | Useful for debug, but communicating the output slows things down

In [7]:
k = 100
queryDuration = 120
spikesPerQuery = 240
neuronThreshold = 64000
benchmark = True

### Setup dimensionality reduction
We use a projection matrix to reduce the dimensionality of the dataset.  
If the projection matrix has already been saved as a file (encodingFile) it will be loaded, otherwise it will be created from the dataset (can take a while)

In [8]:
knnDataset = readGistData(datasetFile)
dr = DR(encodingFile=encodingFile, dataset=knnDataset)

### Setup the dataset
We can fit 2400 dataset samples per chip.  
The code below reduces the number of samples in the dataset to the size we can fit on the system, and applies dimensionality reduction.  
"systemNum" can be used to split the dataset across multiple systems. The n-th system would be run by setting systemNum=n. For example, running on Pohoiki columns 0/1/2 would be achieved by setting systemNum = 0/1/2

In [9]:
systemNum = 0
datasetSize = 2400*numChips
knnDataset = knnDataset[np.arange(systemNum*datasetSize, (systemNum+1)*datasetSize), :]
reducedDataset = dr.reduce(knnDataset)

### Setup the KNN module
This sets up the board object and starts it running. This takes about 5 mins for Nahuku, and multiples of 5 mins for multiple Nahukus (Pohoiki).

In [10]:
knn = Knn(reducedDataset, neuronThreshold, k, queryDuration, spikesPerQuery, benchmark)

### Load the queries and query the KNN module
Although queries are passed as an ndarray, the test function processes them as if they were arriving one at a time. The steps run by "test()" for each query are:
* Dimensionality reduction
* Encoding as spikes
* Sending to the Host
* Running on the Loihi system
* Receiving the results from the Host
* Tie breaking on the superhost (if there is a multiway tie for K'th place)

The returned parameters are:
* results: An array of results with "n" rows and "K" columns
* queryEndTime: The time at which the K'th solution was found on Loihi for each query (An array of "n" times. The difference between elements gives the query duration)
* numSteps: The number of Loihi timesteps before the K'th solution was found for each query (Array of "n" timestep measurements). Only populate if benchmark==True.  
* timePerTimestep: A list of "n" arrays, with each array containing timestep durations measured in microseconds. Calculated by measuring the time at the beginning of each spiking phase and taking the differences. The first array element is the time since the start of the last timestep of the previous sample and therefore includes system reset and communication of result back to the host. This variable is only populated if benchmark==True

Programming the system takes approximately 1 minute per Nahuku board

In [11]:
queries = readGistData(queryFile)
numQueryIndices = queries.shape[0]
results, queryEndTime, numSteps, timePerTimestep =  knn.test(queries, knnDataset, encodingFunction=dr.reduce)

The call below tells the module we are done and that it can disconnect from the Loihi system

In [12]:
knn.finish()

### Compute ground truth and accuracy
For each query, find the closest K dataset samples using the cosine similarity distance.  
For each query, compute what percentage of the top K dataset samples were found by Loihi.

In [13]:
groundTruth = knn.computeGroundTruth(knnDataset, queries, k)
accuracy = np.array([np.intersect1d(groundTruth[ii], res, assume_unique=True, return_indices=False).shape[0] for ii,res in enumerate(results)])/k

### Print some statistics
The embedded x86 cores on Loihi record the time at which the k'th solution was found for each query (queryEndTime).  
Loihi time per query is calculated by taking the difference of these times. i.e. it is measured as the time between finding the k'th solution for subsequent queries, including time required to receive a query, return the result, and reset the Loihi system in between queries.  
The knn module on the superhost first sends (n="knn.pipelining") queries to the host without receiving any results.  
Thereafter, the knn module must receive a result from the host to make space in the channel buffer before it can send another query.  
In the inital phase (first "knn.pipelining" queries) the Loihi system runs quickly, but thereafter the speed is limited by the superhost.

The other time measurements for the superhost are:


| Description         | Includes                       |
|---------------------|--------------------------------|
|Pre-processing       | Dimensionality reduction, normalization, sorting to find the strongest components, converting to spikes, encoding spikes as a lakemont message |
|Send a query         | Communicating the lakemont message from superhost to host (network latency) |
|Receive a result     | Receiving a message containing results from the host (network latency) |
|Post-process result  | Interpreting the Lakemont message and computing ground truth distance for any tied results                               |
|Receive timing info  | Receiving precise timing information (timePerTimestep) from the host (network latency)            |
|Post-process timing  | Interpreting the timing message                               |

The bottleneck is network latency in the superhost to host communication.  
Pre- and Post-processing times are heavily affected by the current machine load from other users.

"numSteps" and "timePerTimestep" are only populated if benchmark==True.   
numSteps is an array with length equal to the number of queries, indicating how many Loihi timesteps each query took.  
timePerTimestep is a list of arrays, one array per query, where each array element indicates the difference in time between the start of the current timestep and previous timestep. The n-th array will have length numSteps[n]. The first element in each array is the time since the beginning of the last timestep of the previous query and includes all the time taken to communicate results to the host, receive a new query, and reset the system back to its initial state.

In [14]:
timePerSample = np.diff(queryEndTime)
print("KNN results for params:\nk\t\t{}\nqueryDuration\t{}\nspikesPerQuery\t{}\nneuronThreshold\t{}\ndatasetSize\t{}".format(k, 
                                                                                                                          queryDuration, 
                                                                                                                          spikesPerQuery, 
                                                                                                                          neuronThreshold,
                                                                                                                          datasetSize))
print("\nAccuracy: \t\t mean {:.2f}%\t min {:.2f}%\t max {:.2f}%".format(np.mean(accuracy)*100, 
                                                                              np.min(accuracy)*100, 
                                                                              np.max(accuracy)*100))
print("Loihi time per query:\t mean {:.2f}ms\t min {:.2f}ms\t max {:.2f}ms".format(np.mean(timePerSample)*1000, 
                                                                                   np.min(timePerSample)*1000, 
                                                                                   np.max(timePerSample)*1000))
print("(first {} queries:\t mean {:.2f}ms\t min {:.2f}ms\t max {:.2f}ms)".format(knn.pipelining, 
                                                                                 np.mean(timePerSample[:knn.pipelining])*1000, 
                                                                                 np.min(timePerSample[:knn.pipelining])*1000, 
                                                                                 np.max(timePerSample[:knn.pipelining])*1000))

print("\nAverage pre-processing time per query:\t\t{:.2f}ms".format(1000*knn.queryPreProcTime/numQueryIndices))
print("Average time to send a query to the host: \t{:.2f}ms".format(1000*knn.querySendTime/numQueryIndices))
print("Average time to receive a result from the host:\t{:.2f}ms".format(1000*knn.resultRecTime/numQueryIndices))
print("Average post-processing time on the superhost \t{:.2f}ms".format(1000*knn.resultProcTime/numQueryIndices))

if benchmark:
    print("\nAdditional Timing info:")
    print("Timesteps per query:\t mean {:.2f}\t min {:.2f}\t max {:.2f}".format(np.mean(numSteps), 
                                                                                np.min(numSteps), 
                                                                                np.max(numSteps)))

    print("Receiving per timestep timing information averaged \t\t{:.2f}ms per query".format(1000*knn.timingRecTime/numQueryIndices))
    print("Post-processing per timestep timing information averaged \t{:.2f}ms per query".format(knn.timingProcTime/numQueryIndices))

KNN results for params:
k		100
queryDuration	120
spikesPerQuery	240
neuronThreshold	64000
datasetSize	76800

Accuracy: 		 mean 87.60%	 min 1.00%	 max 100.00%
Loihi time per query:	 mean 9.92ms	 min 1.63ms	 max 65.59ms
(first 64 queries:	 mean 2.14ms	 min 1.63ms	 max 9.69ms)

Average pre-processing time per query:		0.34ms
Average time to send a query to the host: 	1.87ms
Average time to receive a result from the host:	3.92ms
Average post-processing time on the superhost 	0.63ms

Additional Timing info:
Timesteps per query:	 mean 135.50	 min 110.00	 max 170.00
Receiving per timestep timing information averaged 		3.53ms per query
Post-processing per timestep timing information averaged 	0.00ms per query
